In [1]:
# GPT with covernet input

import importlib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np

from nuscenes.prediction.models.backbone import ResNetBackbone

import utilsHannes as utilsH
from utilsHannes import CoverNetNoRelu
from utilsHannes import ConstantLatticeLoss
from utilsHannes import mean_pointwise_l2_distance
importlib.reload(utilsH)

#################################################################################################################################
# Define your custom dataset class that inherits from torch.utils.data.Dataset
class NuscenesDataset(Dataset):
    def __init__(self, image_data, agent_state_data, ground_truth_data):
        self.image_data = image_data
        self.agent_state_data = agent_state_data
        self.ground_truth_data = ground_truth_data
        
    def __len__(self):
        return len(self.image_data)
    
    def __getitem__(self, index):
        image_data_item = self.image_data[index]
        agent_state_data_item = self.agent_state_data[index]
        ground_truth_data_item = self.ground_truth_data[index]
        
        return image_data_item, agent_state_data_item, ground_truth_data_item

#################################################################################################################################

version = "v1.0-mini" # v1.0-mini, v1.0-trainval
DATAROOT = "data/sets/nuscenes"
seconds_of_history_used = 2.0


In [2]:

# Get training data
train_subset = "mini_train" # 'mini_train', 'mini_val', 'train', 'val'
train_img_list, train_img_tensor_list, train_agent_state_vector_list, train_future_xy_local_list = utilsH.get_and_format_data(version, DATAROOT, train_subset, seconds_of_history_used)

# Get validation data
val_subset = "mini_val" # 'mini_train', 'mini_val', 'train', 'val'.
val_img_list, val_img_tensor_list, val_agent_state_vector_list, val_future_xy_local_list = utilsH.get_and_format_data(version, DATAROOT, val_subset, seconds_of_history_used)


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.533 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


/opt/conda/envs/nuscenesDev/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
/opt/conda/envs/nuscenesDev/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1824: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.689 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [9]:
# For testing
train_short_size = 100
short_train_img_tensor_list = train_img_tensor_list[:train_short_size]
short_train_agent_state_vector_list = train_agent_state_vector_list[:train_short_size]
short_train_future_xy_local_list = train_future_xy_local_list[:train_short_size]
val_short_size = 25
short_val_img_tensor_list = val_img_tensor_list[:val_short_size]
short_val_agent_state_vector_list = val_agent_state_vector_list[:val_short_size]
short_val_future_xy_local_list = val_future_xy_local_list[:val_short_size]


# Prints
train_num_datapoints = len(train_img_tensor_list)
print(f"train_num_datapoints whole dataset = {train_num_datapoints}")
train_num_datapoints = len(short_train_img_tensor_list)
print(f"train_num_datapoints short = {train_num_datapoints}")
print(f"train_img_tensor_list size = {train_img_tensor_list[0].size()}")
print(f"train_agent_state_vector_list[0] = {train_agent_state_vector_list[0]}")
print(f"train_future_xy_local_list[0][0] = {train_future_xy_local_list[0][0]}\n")
val_num_datapoints = len(val_img_tensor_list)
print(f"val_num_datapoints whole dataset = {val_num_datapoints}")
val_num_datapoints = len(short_val_img_tensor_list)
print(f"val_num_datapoints short = {val_num_datapoints}")
print(f"val_img_tensor_list size = {val_img_tensor_list[0].size()}")
print(f"val_agent_state_vector_list[0] = {val_agent_state_vector_list[0]}")
print(f"val_future_xy_local_list[0][0] = {val_future_xy_local_list[0][0]}\n")


# Variables
batch_size = 10
shuffle = True # Set to True if you want to shuffle the data in the dataloader
num_modes = 64 # 2206, 415, 64 (match with eps_traj_set)
eps_traj_set = 8 # 2, 4, 8 (match with num_modes)
learning_rate = 1e-4 # From Covernet paper: fixed learning rate of 1e−4
num_epochs = 100 

# Define datasets
train_dataset = NuscenesDataset(train_img_tensor_list, train_agent_state_vector_list, train_future_xy_local_list)
train_shortDataset = NuscenesDataset(short_train_img_tensor_list, short_train_agent_state_vector_list, short_train_future_xy_local_list)
val_dataset = NuscenesDataset(val_img_tensor_list, val_agent_state_vector_list, val_future_xy_local_list)
val_shortDataset = NuscenesDataset(short_val_img_tensor_list, short_val_agent_state_vector_list, short_val_future_xy_local_list)

# Instantiate dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
train_shortDataloader = DataLoader(train_shortDataset, batch_size=batch_size, shuffle=shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)
val_shortDataloader = DataLoader(val_shortDataset, batch_size=batch_size, shuffle=shuffle)

# Initialize the CoverNet model
backbone = ResNetBackbone('resnet50') 
covernet = CoverNetNoRelu(backbone, num_modes)

# Lattice and similarity function
with open(f'data/sets/nuscenes-prediction-challenge-trajectory-sets/epsilon_{eps_traj_set}.pkl', 'rb') as f:
    lattice = np.array(pickle.load(f))
similarity_function = mean_pointwise_l2_distance

# Define your loss function and optimizer
loss_function = ConstantLatticeLoss(lattice, similarity_function)
optimizer = optim.Adam(covernet.parameters(), lr=learning_rate) 

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
covernet.to(device)


# Squeeze for correct dimensions
for i, train_img_tensor in enumerate(train_img_tensor_list):
    train_img_tensor_list[i] = torch.squeeze(train_img_tensor, dim=0)
    train_agent_state_vector_list[i] = torch.squeeze(train_agent_state_vector_list[i], dim=0)
    
for j, val_img_tensor in enumerate(val_img_tensor_list):
    val_img_tensor_list[j] = torch.squeeze(val_img_tensor, dim=0)
    val_agent_state_vector_list[j] = torch.squeeze(val_agent_state_vector_list[j], dim=0)


# Training starts
print("Training starts:")

# Training and validation loop
for epoch in range(num_epochs):
    
    # TRAINING
    train_epochLoss = 0
    train_total = 0
    train_correct = 0
    for train_batchCount, train_batch in enumerate(train_shortDataloader):

        # Get train_batch data
        image_tensor, agent_state_vector, ground_truth_trajectory = train_batch
        
        # Send to device
        image_tensor = image_tensor.to(device)
        agent_state_vector = agent_state_vector.to(device)
        ground_truth_trajectory = ground_truth_trajectory.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = covernet(image_tensor, agent_state_vector)

        # Compute loss
        loss = loss_function(logits, ground_truth_trajectory)
        train_epochLoss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()
        
       # Compute accuracy
        for logit, ground_truth in zip(logits, ground_truth_trajectory):
            _, predicted = torch.max(logit, 0)
            closest_lattice_trajectory = similarity_function(torch.Tensor(lattice).to(device), ground_truth)
            train_total += 1
            train_correct += (predicted == closest_lattice_trajectory).sum().item()

        # Print loss for this train_batch
        # print(f"train_batch [{train_batchCount+1}/{int(train_num_datapoints/batch_size)+1}], Batch Loss: {loss.item():.4f}")
     
    
    # VALIDATION
    val_epochLoss = 0
    val_total = 0
    val_correct = 0
    for val_batchCount, val_batch in enumerate(val_shortDataloader):

        # Get val_batch data
        image_tensor, agent_state_vector, ground_truth_trajectory = val_batch

        # Send to device
        image_tensor = image_tensor.to(device)
        agent_state_vector = agent_state_vector.to(device)
        ground_truth_trajectory = ground_truth_trajectory.to(device)
        
        # Forward pass
        logits = covernet(image_tensor, agent_state_vector)
        
        # Compute loss
        loss = loss_function(logits, ground_truth_trajectory)
        val_epochLoss += loss.item()
        
        # Compute accuracy
        for logit, ground_truth in zip(logits, ground_truth_trajectory):
            _, predicted = torch.max(logit, 0)
            closest_lattice_trajectory = similarity_function(torch.Tensor(lattice).to(device), ground_truth)
            val_total += 1
            val_correct += (predicted == closest_lattice_trajectory).sum().item()

        # Print loss for this val_batch
        # print(f"val_batch [{val_batchCount+1}/{int(val_num_datapoints/batch_size)+1}], Batch Loss: {loss.item():.4f}")
     
    # Print losses for this epoch
    print(f"Epoch loss [{epoch+1}/{num_epochs}]: Training: {train_epochLoss:.3f} | Validation: {val_epochLoss:.3f}")
    print(f"Epoch accu [{epoch+1}/{num_epochs}]: Training: {train_correct/train_total*100:.1f} % | Validation: {val_correct/val_total*100:.1f} %\n")

    
# Training complete
print("Training complete!")


train_num_datapoints whole dataset = 49
train_num_datapoints short = 49
train_img_tensor_list size = torch.Size([3, 500, 500])
train_agent_state_vector_list[0] = tensor([2.7443, 1.5398, 0.0000])
train_future_xy_local_list[0][0] = [-0.05469174  1.92462069]

val_num_datapoints whole dataset = 4
val_num_datapoints short = 4
val_img_tensor_list size = torch.Size([3, 500, 500])
val_agent_state_vector_list[0] = tensor([ 0.1485, -0.0008,  0.0000])
val_future_xy_local_list[0][0] = [0.00279729 0.21127512]

Training starts:
Epoch loss [1/100]: Training: 20.212 | Validation: 4.700
Epoch accu [1/100]: Training: 18.4 % | Validation: 0.0 %

Epoch loss [2/100]: Training: 16.417 | Validation: 6.327
Epoch accu [2/100]: Training: 12.2 % | Validation: 0.0 %



KeyboardInterrupt: 